In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

In [2]:
train = pd.read_csv(r"C:\Users\lizac\Anaconda3\train.csv")
test = pd.read_csv(r"C:\Users\lizac\Anaconda3\test.csv")

In [4]:
stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [5]:

test['Is_Response'] = np.nan


In [6]:
user_id = test['User_ID']

In [7]:
def separete (data):
    data['Description'] = data['Description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))
    
    countvec = TfidfVectorizer(analyzer='word', ngram_range = (1,1), min_df=150, max_features=500)
    bagofwords = countvec.fit_transform(data['Description'])
    bow_df = pd.DataFrame(bagofwords.todense())
    bow_df.columns = ['col'+ str(x) for x in bow_df.columns]
    cols = ['Browser_Used','Device_Used']

    for x in cols:
        lbl = LabelEncoder()
        data[x] = lbl.fit_transform(data[x])
    data1 = pd.concat([data[cols],data['Is_Response'],bow_df], axis = 1)
    
    return data1
    

In [8]:
test=separete(test)

In [9]:
train=separete(train)

In [11]:
train['Is_Response'] = [1 if x == 'happy' else 0 for x in train['Is_Response']]

In [12]:
x_train= train.drop('Is_Response', axis=1)

In [13]:
y_train=train['Is_Response']

In [14]:
x_test=test.drop('Is_Response', axis=1)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [145]:
param_grid = {"max_depth": [2,3,4,5,6,7,8,9,10,11,12]
            ,"min_samples_split" :[2,3,4,5,6]
            ,"n_estimators" : [10]
            ,"min_samples_leaf": [1,2,3,4,5]
            ,"max_features": (4,5,6,"sqrt")
            ,"criterion": ('gini','entropy')}
rf_regr = RandomForestClassifier()
grid = GridSearchCV(rf_regr, param_grid, cv=StratifiedKFold(),
                   verbose=8, scoring='f1_weighted', n_jobs=-1)

In [115]:
rf_regr.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [116]:
preds = rf_regr.predict(x_test)

In [146]:
from catboost import CatBoostClassifier,cv, Pool

In [147]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.03,rsm = 0.3 ,depth=6, eval_metric='Accuracy', random_seed=2017)

In [148]:
model.fit(x_train, y_train)

0:	learn: 0.7486900	total: 111ms	remaining: 1m 50s
1:	learn: 0.7667985	total: 220ms	remaining: 1m 49s
2:	learn: 0.7801808	total: 333ms	remaining: 1m 50s
3:	learn: 0.7862941	total: 451ms	remaining: 1m 52s
4:	learn: 0.7891452	total: 588ms	remaining: 1m 56s
5:	learn: 0.7894020	total: 713ms	remaining: 1m 58s
6:	learn: 0.7896332	total: 829ms	remaining: 1m 57s
7:	learn: 0.7925100	total: 949ms	remaining: 1m 57s
8:	learn: 0.7915596	total: 1.06s	remaining: 1m 57s
9:	learn: 0.7938457	total: 1.18s	remaining: 1m 57s
10:	learn: 0.7938200	total: 1.3s	remaining: 1m 57s
11:	learn: 0.7937429	total: 1.41s	remaining: 1m 56s
12:	learn: 0.7955923	total: 1.53s	remaining: 1m 56s
13:	learn: 0.7972105	total: 1.65s	remaining: 1m 56s
14:	learn: 0.7967225	total: 1.79s	remaining: 1m 57s
15:	learn: 0.7969280	total: 1.91s	remaining: 1m 57s
16:	learn: 0.7981352	total: 2.03s	remaining: 1m 57s
17:	learn: 0.7995736	total: 2.15s	remaining: 1m 57s
18:	learn: 0.8001387	total: 2.26s	remaining: 1m 56s
19:	learn: 0.8004212	to

158:	learn: 0.8464245	total: 19.8s	remaining: 1m 44s
159:	learn: 0.8466043	total: 19.9s	remaining: 1m 44s
160:	learn: 0.8467328	total: 20s	remaining: 1m 44s
161:	learn: 0.8471951	total: 20.1s	remaining: 1m 44s
162:	learn: 0.8473749	total: 20.3s	remaining: 1m 44s
163:	learn: 0.8474520	total: 20.4s	remaining: 1m 43s
164:	learn: 0.8476318	total: 20.5s	remaining: 1m 43s
165:	learn: 0.8478373	total: 20.7s	remaining: 1m 43s
166:	learn: 0.8480684	total: 20.8s	remaining: 1m 43s
167:	learn: 0.8482225	total: 20.9s	remaining: 1m 43s
168:	learn: 0.8484794	total: 21s	remaining: 1m 43s
169:	learn: 0.8484794	total: 21.1s	remaining: 1m 43s
170:	learn: 0.8488390	total: 21.3s	remaining: 1m 43s
171:	learn: 0.8489931	total: 21.4s	remaining: 1m 42s
172:	learn: 0.8494811	total: 21.5s	remaining: 1m 42s
173:	learn: 0.8497123	total: 21.6s	remaining: 1m 42s
174:	learn: 0.8499949	total: 21.8s	remaining: 1m 42s
175:	learn: 0.8502517	total: 21.9s	remaining: 1m 42s
176:	learn: 0.8503031	total: 22s	remaining: 1m 42s

314:	learn: 0.8679749	total: 39s	remaining: 1m 24s
315:	learn: 0.8682575	total: 39.1s	remaining: 1m 24s
316:	learn: 0.8681804	total: 39.3s	remaining: 1m 24s
317:	learn: 0.8680777	total: 39.4s	remaining: 1m 24s
318:	learn: 0.8681547	total: 39.6s	remaining: 1m 24s
319:	learn: 0.8681804	total: 39.7s	remaining: 1m 24s
320:	learn: 0.8682061	total: 39.8s	remaining: 1m 24s
321:	learn: 0.8682575	total: 40s	remaining: 1m 24s
322:	learn: 0.8684886	total: 40.1s	remaining: 1m 24s
323:	learn: 0.8684630	total: 40.2s	remaining: 1m 23s
324:	learn: 0.8684886	total: 40.4s	remaining: 1m 23s
325:	learn: 0.8686684	total: 40.5s	remaining: 1m 23s
326:	learn: 0.8688739	total: 40.7s	remaining: 1m 23s
327:	learn: 0.8687969	total: 40.8s	remaining: 1m 23s
328:	learn: 0.8689767	total: 40.9s	remaining: 1m 23s
329:	learn: 0.8691051	total: 41s	remaining: 1m 23s
330:	learn: 0.8692335	total: 41.2s	remaining: 1m 23s
331:	learn: 0.8692335	total: 41.3s	remaining: 1m 23s
332:	learn: 0.8692335	total: 41.4s	remaining: 1m 22s

470:	learn: 0.8787373	total: 58.5s	remaining: 1m 5s
471:	learn: 0.8787887	total: 58.6s	remaining: 1m 5s
472:	learn: 0.8787887	total: 58.8s	remaining: 1m 5s
473:	learn: 0.8788657	total: 58.9s	remaining: 1m 5s
474:	learn: 0.8789428	total: 59s	remaining: 1m 5s
475:	learn: 0.8790455	total: 59.1s	remaining: 1m 5s
476:	learn: 0.8790712	total: 59.2s	remaining: 1m 4s
477:	learn: 0.8789685	total: 59.4s	remaining: 1m 4s
478:	learn: 0.8790712	total: 59.5s	remaining: 1m 4s
479:	learn: 0.8790712	total: 59.6s	remaining: 1m 4s
480:	learn: 0.8790712	total: 59.7s	remaining: 1m 4s
481:	learn: 0.8791739	total: 59.8s	remaining: 1m 4s
482:	learn: 0.8792510	total: 60s	remaining: 1m 4s
483:	learn: 0.8793794	total: 1m	remaining: 1m 4s
484:	learn: 0.8794822	total: 1m	remaining: 1m 3s
485:	learn: 0.8793281	total: 1m	remaining: 1m 3s
486:	learn: 0.8794822	total: 1m	remaining: 1m 3s
487:	learn: 0.8794565	total: 1m	remaining: 1m 3s
488:	learn: 0.8795592	total: 1m	remaining: 1m 3s
489:	learn: 0.8795079	total: 1m	re

628:	learn: 0.8851074	total: 1m 17s	remaining: 46s
629:	learn: 0.8852615	total: 1m 18s	remaining: 45.9s
630:	learn: 0.8851587	total: 1m 18s	remaining: 45.7s
631:	learn: 0.8852872	total: 1m 18s	remaining: 45.6s
632:	learn: 0.8854670	total: 1m 18s	remaining: 45.5s
633:	learn: 0.8855183	total: 1m 18s	remaining: 45.4s
634:	learn: 0.8854670	total: 1m 18s	remaining: 45.3s
635:	learn: 0.8853642	total: 1m 18s	remaining: 45.1s
636:	learn: 0.8853385	total: 1m 19s	remaining: 45s
637:	learn: 0.8853385	total: 1m 19s	remaining: 44.9s
638:	learn: 0.8852872	total: 1m 19s	remaining: 44.8s
639:	learn: 0.8853129	total: 1m 19s	remaining: 44.7s
640:	learn: 0.8852615	total: 1m 19s	remaining: 44.5s
641:	learn: 0.8851844	total: 1m 19s	remaining: 44.4s
642:	learn: 0.8852872	total: 1m 19s	remaining: 44.3s
643:	learn: 0.8852358	total: 1m 19s	remaining: 44.2s
644:	learn: 0.8851587	total: 1m 20s	remaining: 44.1s
645:	learn: 0.8851844	total: 1m 20s	remaining: 43.9s
646:	learn: 0.8854413	total: 1m 20s	remaining: 43.

784:	learn: 0.8895510	total: 1m 37s	remaining: 26.8s
785:	learn: 0.8895510	total: 1m 37s	remaining: 26.6s
786:	learn: 0.8894483	total: 1m 37s	remaining: 26.5s
787:	learn: 0.8896538	total: 1m 38s	remaining: 26.4s
788:	learn: 0.8899877	total: 1m 38s	remaining: 26.3s
789:	learn: 0.8899620	total: 1m 38s	remaining: 26.1s
790:	learn: 0.8899877	total: 1m 38s	remaining: 26s
791:	learn: 0.8900647	total: 1m 38s	remaining: 25.9s
792:	learn: 0.8901161	total: 1m 38s	remaining: 25.8s
793:	learn: 0.8901675	total: 1m 38s	remaining: 25.6s
794:	learn: 0.8902959	total: 1m 38s	remaining: 25.5s
795:	learn: 0.8901932	total: 1m 39s	remaining: 25.4s
796:	learn: 0.8902188	total: 1m 39s	remaining: 25.3s
797:	learn: 0.8901932	total: 1m 39s	remaining: 25.1s
798:	learn: 0.8902702	total: 1m 39s	remaining: 25s
799:	learn: 0.8902445	total: 1m 39s	remaining: 24.9s
800:	learn: 0.8901932	total: 1m 39s	remaining: 24.8s
801:	learn: 0.8900904	total: 1m 39s	remaining: 24.6s
802:	learn: 0.8901932	total: 1m 39s	remaining: 24.

940:	learn: 0.8939176	total: 1m 57s	remaining: 7.34s
941:	learn: 0.8938919	total: 1m 57s	remaining: 7.21s
942:	learn: 0.8940203	total: 1m 57s	remaining: 7.09s
943:	learn: 0.8940717	total: 1m 57s	remaining: 6.96s
944:	learn: 0.8940717	total: 1m 57s	remaining: 6.84s
945:	learn: 0.8941745	total: 1m 57s	remaining: 6.71s
946:	learn: 0.8942772	total: 1m 57s	remaining: 6.59s
947:	learn: 0.8943543	total: 1m 57s	remaining: 6.47s
948:	learn: 0.8943286	total: 1m 58s	remaining: 6.34s
949:	learn: 0.8944056	total: 1m 58s	remaining: 6.22s
950:	learn: 0.8944313	total: 1m 58s	remaining: 6.09s
951:	learn: 0.8943799	total: 1m 58s	remaining: 5.97s
952:	learn: 0.8943799	total: 1m 58s	remaining: 5.85s
953:	learn: 0.8943029	total: 1m 58s	remaining: 5.72s
954:	learn: 0.8943799	total: 1m 58s	remaining: 5.6s
955:	learn: 0.8945597	total: 1m 58s	remaining: 5.47s
956:	learn: 0.8945854	total: 1m 59s	remaining: 5.35s
957:	learn: 0.8947395	total: 1m 59s	remaining: 5.22s
958:	learn: 0.8946111	total: 1m 59s	remaining: 

In [149]:
preds = model.predict(x_test)

In [97]:

import lightgbm as lgb

In [98]:

# set the data in format lgb accepts
d_train = lgb.Dataset(x_train, label = y_train)

In [99]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 7, 
    'num_leaves': 21, 
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [100]:
lgb_cv = lgb.cv(params, d_train, 
                num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's binary_error: 0.197396 + 0.00370413
[40]	cv_agg's binary_error: 0.181419 + 0.00339114
[60]	cv_agg's binary_error: 0.1676 + 0.00318024
[80]	cv_agg's binary_error: 0.159817 + 0.0024246
[100]	cv_agg's binary_error: 0.15319 + 0.0026623
[120]	cv_agg's binary_error: 0.148335 + 0.00305291
[140]	cv_agg's binary_error: 0.143969 + 0.00346573
[160]	cv_agg's binary_error: 0.140399 + 0.00344514
[180]	cv_agg's binary_error: 0.137624 + 0.00345125
[200]	cv_agg's binary_error: 0.135852 + 0.00357055
[220]	cv_agg's binary_error: 0.134979 + 0.0029736
[240]	cv_agg's binary_error: 0.133566 + 0.00232499
[260]	cv_agg's binary_error: 0.13277 + 0.00211179
[280]	cv_agg's binary_error: 0.131434 + 0.00170249
[300]	cv_agg's binary_error: 0.130612 + 0.00108126
[320]	cv_agg's binary_error: 0.129688 + 0.00129822
[340]	cv_agg's binary_error: 0.12884 + 0.00153857
[360]	cv_agg's binary_error: 0.128686 + 0.00163185
[380]	cv_agg's binary_error: 0.127992 + 0.00125355
[400]	cv_agg's binary_error: 0.12771 + 0.00

In [101]:
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [102]:

model = lgb.train(params, d_train, num_boost_round=nround)

In [103]:
preds = model.predict(x_test)

In [104]:
def to_labels(x):
    if x > 0.25:
        return "happy"
    return "not_happy"

In [24]:
from sklearn import metrics

In [34]:
Xtrain, Xtest, ytrain, ytest = train_test_split(x_train, y_train, random_state=2)


d_train = lgb.Dataset(Xtrain, label = ytrain)
lgb_cv = lgb.cv(params, d_train, 
                num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

model = lgb.train(params, d_train, num_boost_round=nround)
preds = model.predict(Xtest)

[20]	cv_agg's binary_error: 0.197541 + 0.00287325
[40]	cv_agg's binary_error: 0.180554 + 0.00218896
[60]	cv_agg's binary_error: 0.167335 + 0.00397643
[80]	cv_agg's binary_error: 0.15891 + 0.0031776
[100]	cv_agg's binary_error: 0.151375 + 0.00334571
[120]	cv_agg's binary_error: 0.14682 + 0.00347295
[140]	cv_agg's binary_error: 0.14408 + 0.00386912
[160]	cv_agg's binary_error: 0.141854 + 0.00384356
[180]	cv_agg's binary_error: 0.140039 + 0.00428472
[200]	cv_agg's binary_error: 0.137642 + 0.00353274
[220]	cv_agg's binary_error: 0.136443 + 0.00315489
[240]	cv_agg's binary_error: 0.135313 + 0.00408918
[260]	cv_agg's binary_error: 0.133703 + 0.00414725
[280]	cv_agg's binary_error: 0.132333 + 0.00401936
[300]	cv_agg's binary_error: 0.132025 + 0.00424221
[320]	cv_agg's binary_error: 0.131066 + 0.00446427
[340]	cv_agg's binary_error: 0.130792 + 0.00428621
[360]	cv_agg's binary_error: 0.130039 + 0.0046081
[380]	cv_agg's binary_error: 0.12997 + 0.00440625
[400]	cv_agg's binary_error: 0.129114 + 0

In [27]:
pred1=pd.DataFrame(preds)

In [83]:
a=pd.DataFrame(ytest).reset_index()
d=pd.DataFrame(preds).reset_index()

d=d.drop('index',axis=1)
a=a.drop('index',axis=1)
c=pd.concat([a,d],axis=1)


Index(['Is_Response', 0], dtype='object')

In [96]:
c=c[c['Is_Response']==0]
c[0].median()

0.25775917035798324

In [105]:
sub5 = pd.DataFrame({'User_ID':user_id, 'Is_Response':preds})
sub5['Is_Response'] = sub5['Is_Response'].map(lambda x: to_labels(x))
sub5 = sub5[['User_ID','Is_Response']]
sub5.to_csv(r"C:\Users\lizac\Anaconda3\s2.csv", index=False)